In [1]:
# Make project root importable so `import src...` works
from pathlib import Path
import sys

PROJECT_ROOT = Path().resolve().parents[0]  # one level up from notebooks/
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("Project root on sys.path:", PROJECT_ROOT)


Project root on sys.path: C:\Users\User\Desktop\Projects\Car_Price


In [2]:
from pathlib import Path
import pandas as pd
import numpy as np
import json, joblib

from src.inference import load_artifacts, predict_and_cluster
from src.recommend import load_catalog, recommend_by_budget_only, recommend_minimal

PROJECT_ROOT = Path().resolve().parents[0]
MODELS_DIR   = PROJECT_ROOT / "models"
REPORTS_DIR  = PROJECT_ROOT / "reports" / "results"
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

artifacts = load_artifacts(PROJECT_ROOT)
catalog = load_catalog(PROJECT_ROOT)

print("Loaded:")
print(" - model:", type(artifacts["model"]).__name__)
print(" - features:", len(artifacts["feature_columns"]))
print(" - kmeans feats:", artifacts["kmeans_features"])


Loaded:
 - model: LGBMRegressor
 - features: 37
 - kmeans feats: ['Mileage(km)', 'Year', 'Horsepower', 'EngineSize(L)']


In [4]:
from pathlib import Path
import pandas as pd
from src.inference import load_artifacts, predict_and_cluster

# Load artifacts + set reports dir
PROJECT_ROOT = Path().resolve().parents[0]
artifacts = load_artifacts(PROJECT_ROOT)
REPORTS_DIR = PROJECT_ROOT / "reports" / "results"
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

samples = [
    {
        "Brand": "Hyundai", "Model": "Accent", "Year": 2011, "Condition": "Used",
        "Mileage(km)": 180000, "EngineSize(L)": 1.4, "FuelType": "Gasoline",
        "Horsepower": 100, "Torque": 97, "Transmission": "Manual",
        "DriveType": "FWD", "BodyType": "Sedan",
        "FuelEfficiency(L/100km)": 7.8
    }
,
    {
        "Brand":"Mercedes-Benz","Model":"S-Class","Year":2022,"Condition":"Used",
        "Mileage(km)":5000,"EngineSize(L)":4.0,"FuelType":"Gasoline","Horsepower":496,"Torque":516,
        "Transmission":"Automatic","DriveType":"AWD","BodyType":"Sedan","Doors":4,"Seats":5,
        "AccidentHistory":"No", "FuelEfficiency(L/100km)":7.2,
    },
]

pred_rows = []
for i, s in enumerate(samples, 1):
    try:
        out = predict_and_cluster(s, artifacts)
        assert out["predicted_price"] > 0
        assert isinstance(out["cluster_label"], int)
        print(f"Sample {i} → price=${out['predicted_price']:.0f}, segment={out['cluster_name']}")
        pred_rows.append({
            "sample_id": i,
            "predicted_price": round(out["predicted_price"], 2),
            "cluster_label": out["cluster_label"],
            "cluster_name": out["cluster_name"]
        })
    except Exception as e:
        print(f"Sample {i} failed: {e}")

pred_df = pd.DataFrame(pred_rows)
pred_path = REPORTS_DIR / "smoke_predict_samples.csv"
pred_df.to_csv(pred_path, index=False)
print("Saved:", pred_path)
pred_df


Sample 1 → price=$10822, segment=Budget
Sample 2 → price=$44586, segment=Luxury
Saved: C:\Users\User\Desktop\Projects\Car_Price\reports\results\smoke_predict_samples.csv


,sample_id,predicted_price,cluster_label,cluster_name
0,1,10822.29,0,Budget
1,2,44585.67,0,Luxury
